# Introduction to XGBoost Spark3.0 with GPU

Agaricus is an example of XGBoost classifier for multiple classification. This notebook will show you how to load data, train the xgboost model. Comparing to original XGBoost Spark code, there're only one API difference.

## Load libraries
First load some common libraries will be used by both GPU version and CPU version XGBoost.

In [1]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}

Besides CPU version requires some extra libraries, such as:

```scala
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.functions._
```

## Set the dataset paths

In [2]:
// You need to update them to your real paths!
val dataRoot = sys.env.getOrElse("DATA_ROOT", "/data")
val trainPath = dataRoot + "/agaricus/csv/train/"
val evalPath  = dataRoot + "/agaricus/csv/test/"
val transPath = dataRoot + "/agaricus/csv/test/"

lastException = null
dataRoot = /data
trainPath = /data/agaricus/csv/train/
evalPath = /data/agaricus/csv/test/
transPath = /data/agaricus/csv/test/


/data/agaricus/csv/test/

## Build the schema of the dataset

For agaricus example, the data has 126 dimensions, being named as "feature_0", "feature_1" ... "feature_125". The schema will be used to load data in the future.

In [3]:
val labelName = "label"
def columnNames(length: Int): List[String] =
  0.until(length).map(i => s"feature_$i").toList.+:(labelName)

def schema(length: Int): StructType =
  StructType(columnNames(length).map(n => StructField(n, DoubleType)))

val dataSchema = schema(126)

// Build the column name list for features.
val featureCols = dataSchema.filter(_.name != labelName).map(_.name).toArray

labelName = label
dataSchema = StructType(StructField(label,DoubleType,true), StructField(feature_0,DoubleType,true), StructField(feature_1,DoubleType,true), StructField(feature_2,DoubleType,true), StructField(feature_3,DoubleType,true), StructField(feature_4,DoubleType,true), StructField(feature_5,DoubleType,true), StructField(feature_6,DoubleType,true), StructField(feature_7,DoubleType,true), StructField(feature_8,DoubleType,true), StructField(feature_9,DoubleType,true), StructField(feature_10,DoubleType,true), StructField(feature_11,DoubleType,true), StructField(feature_12,DoubleType,true), StructField(feature_13,DoubleType,true), StructFiel...


columnNames: (length: Int)List[String]
schema: (length: Int)org.apache.spark.sql.types.StructType


StructType(StructField(label,DoubleType,true), StructField(feature_0,DoubleType,true), StructField(feature_1,DoubleType,true), StructField(feature_2,DoubleType,true), StructField(feature_3,DoubleType,true), StructField(feature_4,DoubleType,true), StructField(feature_5,DoubleType,true), StructField(feature_6,DoubleType,true), StructField(feature_7,DoubleType,true), StructField(feature_8,DoubleType,true), StructField(feature_9,DoubleType,true), StructField(feature_10,DoubleType,true), StructField(feature_11,DoubleType,true), StructField(feature_12,DoubleType,true), StructField(feature_13,DoubleType,true), StructFiel...

## Create a new spark session and load data

A new spark session should be created to continue all the following spark operations.

NOTE: in this notebook, the dependency jars have been loaded when installing toree kernel. Alternatively the jars can be loaded into notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called just after a new spark session is created. Do it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("agaricus-GPU").getOrCreate
%AddJar file:/data/libs/rapids-4-spark-XXX.jar
%AddJar file:/data/libs/xgboost4j-spark-gpu_2.12-XXX.jar
%AddJar file:/data/libs/xgboost4j-gpu_2.12-XXX.jar
// ...
```

##### Please note the new jar "rapids-4-spark-XXX.jar" is only needed for GPU version, you can not add it to dependence list for CPU version.

In [4]:
// Build the spark session and data reader as usual
val sparkSession = SparkSession.builder.appName("agaricus-gpu").getOrCreate
val dataReader = sparkSession.read.option("header", true).schema(dataSchema)

sparkSession = org.apache.spark.sql.SparkSession@3886ba44
dataReader = org.apache.spark.sql.DataFrameReader@5c8be07f


org.apache.spark.sql.DataFrameReader@5c8be07f

In [5]:
// load all the dataset
val trainSet = dataReader.csv(trainPath)
val evalSet  = dataReader.csv(evalPath)
val transSet = dataReader.csv(transPath)

trainSet = [label: double, feature_0: double ... 125 more fields]
evalSet = [label: double, feature_0: double ... 125 more fields]
transSet = [label: double, feature_0: double ... 125 more fields]


[label: double, feature_0: double ... 125 more fields]

## Set XGBoost parameters and build a XGBoostClassifier

For CPU version, `num_workers` is recommended being equal to the number of CPU cores, while for GPU version, it should be set to the number of GPUs in Spark cluster.

Besides the `tree_method` for CPU version is also different from that for GPU version. Now only "gpu_hist" is supported for training on GPU.

```scala
// difference in parameters
  "num_workers" -> 12,
  "tree_method" -> "hist",
```

In [6]:
// build XGBoost classifier
val paramMap = Map(
  "num_workers" -> 1,
  "tree_method" -> "gpu_hist",
  "num_round" -> 100
)

paramMap = Map(num_workers -> 1, tree_method -> gpu_hist, num_round -> 100)


Map(num_workers -> 1, tree_method -> gpu_hist, num_round -> 100)

In [7]:
val xgbClassifier  = new XGBoostClassifier(paramMap)
  .setLabelCol(labelName)
  .setFeaturesCol(featureCols)

xgbClassifier = xgbc_57e2d7fc657a


xgbc_57e2d7fc657a

## Benchmark and train
The object `benchmark` is used to compute the elapsed time of some operations.

Training with evaluation sets is also supported in 2 ways, the same as CPU version's behavior:

* Call API `setEvalSets` after initializing an XGBoostClassifier

```scala
xgbClassifier.setEvalSets(Map("eval" -> evalSet))

```

* Use parameter `eval_sets` when initializing an XGBoostClassifier

```scala
val paramMapWithEval = paramMap + ("eval_sets" -> Map("eval" -> evalSet))
val xgbClassifierWithEval = new XGBoostClassifier(paramMapWithEval)
```

Here chooses the API way to set evaluation sets.

In [8]:
xgbClassifier.setEvalSets(Map("eval" -> evalSet))

xgbc_57e2d7fc657a

In [9]:
object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("Elapsed time [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}

defined object Benchmark


In [10]:
// start training
println("\n------ Training ------")
val (xgbClassificationModel, _) = Benchmark.time("train") {
  xgbClassifier.fit(trainSet)
}


------ Training ------
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=34739, DMLC_NUM_WORKER=1}


xgbClassificationModel = xgbc_57e2d7fc657a


Elapsed time [train]: 11.177s


xgbc_57e2d7fc657a

## Transformation and evaluation
Here uses `transSet` to evaluate our model and prints some useful columns to show our prediction result. After that `MulticlassClassificationEvaluator` is used to calculate an overall accuracy of our predictions.

In [11]:
// start transform
println("\n------ Transforming ------")
val (results, _) = Benchmark.time("transform") {
  val ret = xgbClassificationModel.transform(transSet).cache()
  ret.foreachPartition((_: Iterator[_]) => ())
  ret
}
results.select(labelName, "rawPrediction", "probability", "prediction").show(10)

println("\n------Accuracy of Evaluation------")
val evaluator = new MulticlassClassificationEvaluator()
evaluator.setLabelCol(labelName)
val accuracy = evaluator.evaluate(results)

println(s"accuracy == $accuracy")


------ Transforming ------
Elapsed time [transform]: 2.51s
+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-4.4405460357666...|[0.99995559453964...|       0.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
+-----+--------------------+--------------------+----------+
only showing top 10 rows


------Accuracy of Evaluation------
accuracy == 0.9069677632

results = [label: double, feature_0: double ... 128 more fields]
evaluator = MulticlassClassificationEvaluator: uid=mcEval_8f89b3a17d4b, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15
accuracy = 0.9069677632722861


0.9069677632722861

## Save the model to disk and load model
Save the model to disk and then load it to memory. After that use the loaded model to do a new prediction.

In [12]:
xgbClassificationModel.write.overwrite.save(dataRoot + "/model/agaricus")

val modelFromDisk = XGBoostClassificationModel.load(dataRoot + "/model/agaricus")
val (results2, _) = Benchmark.time("transform2") {
  modelFromDisk.transform(transSet)
}
results2.select(labelName, "rawPrediction", "probability", "prediction").show(10)

Elapsed time [transform2]: 0.069s


modelFromDisk = xgbc_57e2d7fc657a
results2 = [label: double, feature_0: double ... 128 more fields]


+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-4.4405460357666...|[0.99995559453964...|       0.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  0.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
|  1.0|[-0.9999903440475...|[9.65595245361328...|       1.0|
+-----+--------------------+--------------------+----------+
only showing top 10 rows



[label: double, feature_0: double ... 128 more fields]

In [13]:
sparkSession.close()